In [ ]:
import codecs
import networkx as nx
import os
import pickle
from networkx.drawing.nx_pydot import write_dot
from os import listdir
from os.path import isfile, join

In [ ]:
# toggle1: Directed, Vertex: lexeme, semty. Edges: rel_sem, cstr
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

for input_file in input_files:
    H = nx.DiGraph()
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').replace(' ','').split('\t')
                if line_elements[23] == '' or line_elements[25] == '' or line_elements[36] == '':
                    continue
                
                vertices1 = list()
                vertices2 = list()
                if '|' in line_elements[23]:
                    vertices1.append(line_elements[3] + '_' + line_elements[23].split('|')[0])
                    vertices1.append(line_elements[3] + '_' + line_elements[23].split('|')[1])
                else:
                    vertices1.append(line_elements[3] + '_' + line_elements[23])
                if '|' in line_elements[25]:
                    vertices2.append(line_elements[6] + '_' + line_elements[25].split('|')[0])
                    vertices2.append(line_elements[6] + '_' + line_elements[25].split('|')[1])
                else:
                    vertices2.append(line_elements[6] + '_' + line_elements[25])
                
                if line_elements[21] == 'as2de' or line_elements[21] == 'as2des':
                    edge_type = line_elements[36] + '_' + line_elements[14] + '-' + line_elements[17]
                    for v1 in vertices1:
                        for v2 in vertices2:
                            H.add_edge(v1, v2, label=edge_type)
                elif line_elements[21] == 'de2as' or line_elements[21] == 'des2as':
                    edge_type = line_elements[36] + '_' + line_elements[17] + '-' + line_elements[14]
                    for v1 in vertices1:
                        for v2 in vertices2:
                            H.add_edge(v2, v1, label=edge_type)
                else:
                    edge_type1 = line_elements[36] + '_' + line_elements[14] + '-' + line_elements[17]
                    edge_type2 = line_elements[36] + '_' + line_elements[17] + '-' + line_elements[14]
                    for v1 in vertices1:
                        for v2 in vertices2:
                            H.add_edge(v1, v2, label=edge_type)
                            H.add_edge(v2, v1, label=edge_type)
    write_dot(H, join('graphs edge_relsem_cstr', input_file.replace('.txt','.dot')))
    H.clear()

In [ ]:
# toggle1: directed, vertex: lexeme, category. edges: orientation, complexite, cstr
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

for input_file in input_files:
    #if input_file != 'F01531 instrumentateur_Nm.txt':
        #continue
    H = nx.DiGraph()
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').replace(' ','').split('\t')
                if line_elements[21] == 'as2de' or line_elements[21] == 'as2des':
                    edge_type = line_elements[19] + '_' + line_elements[14] + '-' + line_elements[17]
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_type)
                elif line_elements[21] == 'de2as' or line_elements[21] == 'des2as':
                    edge_type = line_elements[19] + '_' + line_elements[17] + '-' + line_elements[14]
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_type)
                else:
                    sorted_cstr = sorted([line_elements[14], line_elements[17]])
                    edge_type =  line_elements[19] + '_' + line_elements[21] + '_' + sorted_cstr[0] + '-' + sorted_cstr[1]
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_type)
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_type)
        try:
            if not nx.is_weakly_connected(H):
                print(input_file)
        except:
            print('null graph', input_file)
    write_dot(H, join('graphs edge_complexite_cstr', input_file.replace('.txt','.dot')))
    H.clear()

In [ ]:
# toggle1: for gSpan. Directed, vertex: lexeme, category. edges: orientation, complexite, cstr
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

edge_labels = list()
vertex_labels = list()

f_out = codecs.open('for_gSpan.txt', 'w+', encoding='utf-8')
for input_file in input_files:
    #if input_file != 'F01531 instrumentateur_Nm.txt':
        #continue
    H = nx.DiGraph()
    family_number = input_file.split()[0].replace('F', '')
    f_out.write('t # ' + family_number + '\n')
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').replace(' ','').split('\t')
                
                vertex_a_type = line_elements[8]
                try:
                    vertex_a_label = vertex_labels.index(vertex_a_type)
                except ValueError:
                    vertex_a_label = len(vertex_labels)
                    vertex_labels.append(vertex_a_type)
                vertex_b_type = line_elements[10]
                try:
                    vertex_b_label = vertex_labels.index(vertex_b_type)
                except ValueError:
                    vertex_b_label = len(vertex_labels)
                    vertex_labels.append(vertex_b_type)
                H.add_node(line_elements[3] + '_' + line_elements[8], label=vertex_a_label)
                H.add_node(line_elements[6] + '_' + line_elements[10], label=vertex_b_label)
                    
                orientation = line_elements[21]
                if orientation == 'as2de' or orientation == 'as2des':
                    edge_type = line_elements[19] + '_' + line_elements[14] + '-' + line_elements[17] 
                elif orientation == 'de2as' or orientation == 'des2as':
                    edge_type = line_elements[19] + '_' + line_elements[17] + '-' + line_elements[14] 
                else:  # NA or indirect
                    sorted_cstr = sorted([line_elements[14], line_elements[17]])
                    edge_type = line_elements[19] + '_' + orientation + '_' + sorted_cstr[0] + '-' + sorted_cstr[1]
                try:
                    edge_label = edge_labels.index(edge_type)
                except ValueError:
                    edge_label = len(edge_labels)
                    edge_labels.append(edge_type)
                    
                if orientation == 'as2de' or orientation == 'as2des':
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_label)
                elif orientation == 'de2as' or orientation == 'des2as':
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_label)
                else:
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_label)
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_label)
        node_list = list(H.nodes)
        for node_idx, node in enumerate(node_list):
            f_out.write('v ' + str(node_idx) + ' ' + str(H.nodes[node]['label']) + '\n')
            #print('v ' + str(node_idx) + ' ' + str(H.nodes[node]['label']))
        for edge in list(H.edges):
            f_out.write('e ' + str(node_list.index(edge[0])) + ' ' + str(node_list.index(edge[1])) + ' ' + str(H[edge[0]][edge[1]]['label']) + '\n')
            #print('e ' + str(node_list.index(edge[0])) + ' ' + str(node_list.index(edge[1])) + ' ' + str(H[edge[0]][edge[1]]['label']))
f_out.close()
print(vertex_labels)
print(edge_labels)

In [ ]:
# toggle1: using write_dot for all families, directed, no cstr in edges
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

for input_file in input_files:
    H = nx.DiGraph()
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').split('\t')
                orientation = line_elements[21]
                if orientation == 'as2de' or orientation == 'as2des' or orientation == 'de2as' or orientation == 'des2as':
                    orientation = 'direct'
                edge_type = orientation + '_' + line_elements[19]
                if line_elements[21] == 'as2de' or line_elements[21] == 'as2des':
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_type)
                elif line_elements[21] == 'de2as' or line_elements[21] == 'des2as':
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_type)
                else:
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_type)
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_type)
        try:
            if not nx.is_weakly_connected(H):
                print(input_file)
        except:
            print('null graph', input_file)
    write_dot(H, join('graphs_directed', input_file.replace('.txt','.dot'))

In [ ]:
# toggle1: create directed graph for gSpan input, cstr in vertices
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

edge_labels = list()
vertex_labels = list()

f_out = codecs.open('for_gSpan.txt', 'w+', encoding='utf-8')
for input_file in input_files:
    #if input_file != 'F01531 instrumentateur_Nm.txt':
        #continue
    H = nx.DiGraph()
    family_number = input_file.split()[0].replace('F', '')
    f_out.write('t # ' + family_number + '\n')
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').split('\t')
                
                vertex_a_type = line_elements[8] + '_' + line_elements[14]
                try:
                    vertex_a_label = vertex_labels.index(vertex_a_type)
                except ValueError:
                    vertex_a_label = len(vertex_labels)
                    vertex_labels.append(vertex_a_type)
                vertex_b_type = line_elements[10] + '_' + line_elements[17]
                try:
                    vertex_b_label = vertex_labels.index(vertex_b_type)
                except ValueError:
                    vertex_b_label = len(vertex_labels)
                    vertex_labels.append(vertex_b_type)
                H.add_node(line_elements[3] + '_' + line_elements[8], label=vertex_a_label)
                H.add_node(line_elements[6] + '_' + line_elements[10], label=vertex_b_label)
                    
                orientation = line_elements[21]
                if orientation == 'as2de' or orientation == 'as2des' or orientation == 'de2as' or orientation == 'des2as':
                    orientation = 'direct'
                edge_type = orientation + '_' + line_elements[19]
                try:
                    edge_label = edge_labels.index(edge_type)
                except ValueError:
                    edge_label = len(edge_labels)
                    edge_labels.append(edge_type)
                    
                if line_elements[21] == 'as2de' or line_elements[21] == 'as2des':
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_label)
                elif line_elements[21] == 'de2as' or line_elements[21] == 'des2as':
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_label)
                else:
                    H.add_edge(line_elements[3] + '_' + line_elements[8], line_elements[6] + '_' + line_elements[10], label=edge_label)
                    H.add_edge(line_elements[6] + '_' + line_elements[10], line_elements[3] + '_' + line_elements[8], label=edge_label)
        node_list = list(H.nodes)
        for node_idx, node in enumerate(node_list):
            f_out.write('v ' + str(node_idx) + ' ' + str(H.nodes[node]['label']) + '\n')
            #print('v ' + str(node_idx) + ' ' + str(H.nodes[node]['label']))
        for edge in list(H.edges):
            f_out.write('e ' + str(node_list.index(edge[0])) + ' ' + str(node_list.index(edge[1])) + ' ' + str(H[edge[0]][edge[1]]['label']) + '\n')
f_out.close()
print(vertex_labels)
print(edge_labels)

In [ ]:
vertex_label_file = open('label_vertex.p', 'wb')
pickle.dump(vertex_labels, vertex_label_file)
vertex_label_file.close()

edge_label_file = open('label_edge.p', 'wb')
pickle.dump(edge_labels, edge_label_file)
edge_label_file.close()